<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=2fe0da9dd69beec64fada78371d83970b8671c0f337bc24bfc72f22dce31883d
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-10 09:23:38
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  11.04 L
Current:  1.41 C
-------------------
Today PnL: 59.32 K (0.42%)
Current PnL: -18.32 L (-12.32%)
CY Booked + Current PnL: -7.29 L (-4.9%)
-------------------
Total profit:  1.87 L
Total loss:  -20.19 L
-------------------
Total Booked + Current PnL: 19.69 L (16.18%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.83%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 77.97 L (55.34%)
Deployed:  1.22 C
Current:  1.41 C
CAGR/XIRR %: 9.18%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,APOLLOHOSP,8285.00,-13.42,61.0,H-LC,11.16,172964.0,16787.0,9305.0,0.47,10.75,5.38,16.71,22.0,1.80,1.33,30.06,X40N,BTT,HEALTHCARE
77,TTKPRESTIG,770.00,105.55,62.0,M-SC,5.57,88569.0,-12208.0,12302.0,-0.38,-12.11,13.89,0.09,245.0,-0.99,0.68,16.36,OX40N,NTT,DURABLES
62,SBIN,863.00,-13.25,53.0,M-LC,4.25,215604.0,14124.0,13087.0,0.59,7.01,6.07,13.51,60.0,1.08,1.66,20.49,XY25,NTT,BANKS
41,ITC,452.00,-37.92,51.0,H-LC,3.29,200539.0,401.0,20495.0,0.01,0.20,10.22,10.44,4.0,0.02,1.54,5.77,X40,NTT,FMCG
49,NATIONALUM,247.44,-44.49,66.0,H-MC,2.81,110362.0,9858.0,20781.0,0.78,9.81,18.83,30.49,79.0,0.47,0.85,45.39,MH,ATH,METALS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,41.94,61.0,M-MC,7.06,223412.0,-1550.0,169212.0,0.22,-0.69,75.74,74.53,192.0,-0.01,1.72,31.34,XY24,BTT,STEEL
35,ICICIPRULI,790.00,-21.11,40.0,H-MC,2.01,135724.0,-64.0,42821.0,0.40,-0.05,31.55,31.48,107.0,-0.00,1.05,12.07,X40,ATH,INSURANCE
30,HAVELLS,2069.16,0.06,62.0,H-MC,1.74,248040.0,309.0,74759.0,0.18,0.12,30.14,30.30,92.0,0.00,1.91,13.43,X40,ATH,ELECTRICAL
34,ICICIGI,2260.25,-20.58,43.0,H-MC,3.48,136745.0,612.0,30508.0,0.71,0.45,22.31,22.86,91.0,0.02,1.05,13.12,X40,ATH,INSURANCE
51,PGHH,17973.08,-30.68,53.0,H-MC,4.69,200640.0,-180.0,68960.0,-0.10,-0.09,34.37,34.25,80.0,-0.00,1.54,5.10,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,JIOFIN,387.00,-2.53,38.0,H-LC,13.87,215565.0,-1841.0,55336.0,-1.01,-0.85,25.67,24.61,48.0,-0.03,1.66,53.51,XY24,BTT,FINANCE
31,HCLTECH,1943.91,-1.94,38.0,H-LC,10.17,224055.0,-17869.0,81130.0,1.72,-7.39,36.21,26.15,8.0,-0.22,1.73,5.88,X40,ATH,IT
83,VBL,671.64,-15.07,42.0,H-LC,6.37,305698.0,-10144.0,122799.0,0.98,-3.21,40.17,35.67,5.0,-0.08,2.35,10.20,X40N,ATH,FMCG
74,TCS,4998.00,-28.04,43.0,H-LC,14.19,286644.0,-59312.0,183166.0,1.01,-17.14,63.90,35.80,1.0,-0.32,2.21,1.55,X40,BTT,IT
80,UNITDSPR,1644.00,-14.23,46.0,H-LC,8.26,172524.0,-9235.0,44477.0,0.20,-5.08,25.78,19.39,86.0,-0.21,1.33,2.65,X40N,NTT,BREWERIES


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.20,-3.18,60.0,H-LC,1.32,239760.0,-6391.0,85594.0,2.75,-2.60,35.70,32.18,16.0,-0.07,1.85,32.51,X200,ATH,IT
50,NESTLEIND,1377.00,-8.22,60.0,H-LC,2.31,282602.0,17176.0,39621.0,0.46,6.47,14.02,21.40,11.0,0.43,2.18,13.54,XY25,NTT,FMCG
41,ITC,452.00,-37.92,51.0,H-LC,3.29,200539.0,401.0,20495.0,0.01,0.20,10.22,10.44,4.0,0.02,1.54,5.77,X40,NTT,FMCG
20,CIPLA,1795.00,-20.12,51.0,H-LC,5.93,213432.0,8932.0,32484.0,0.41,4.37,15.22,20.25,10.0,0.27,1.64,13.48,X40N,BTT,PHARMA
83,VBL,671.64,-15.07,42.0,H-LC,6.37,305698.0,-10144.0,122799.0,0.98,-3.21,40.17,35.67,5.0,-0.08,2.35,10.20,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TRIDENT,48.00,-7.75,57.0,M-SC,6.03,71852.0,-20402.0,46711.0,3.49,-22.12,65.01,28.51,224.0,-0.44,0.55,22.95,XR,NTT,TEXTILES
47,LTIM,7230.20,-3.18,60.0,H-LC,1.32,239760.0,-6391.0,85594.0,2.75,-2.60,35.70,32.18,16.0,-0.07,1.85,32.51,X200,ATH,IT
86,WIPRO,420.00,-11.98,51.0,M-LC,6.75,154473.0,3528.0,105922.0,2.73,2.34,68.57,72.51,53.0,0.03,1.19,8.50,XR,NTT,IT
16,BSOFT,836.99,-12.86,53.0,H-SC,16.43,67139.0,-42910.0,79331.0,2.62,-38.99,118.16,33.10,131.0,-0.54,0.52,11.56,XR,ATH,IT
22,COFFEEDAY,80.00,-37.63,61.0,L-SC,42.31,91776.0,-21773.0,61820.0,2.38,-19.17,67.36,35.27,268.0,-0.35,0.71,124.62,XR,NTT,HOTELS


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VALIANTORG,838.0,-329.57,58.0,H-SC,8.68,97630.0,-35656.0,120251.0,-3.64,-26.75,123.17,63.47,139.0,-0.30,0.75,63.65,XR,NTT,CHEMICALS
46,LAOPALA,464.0,117.42,53.0,H-SC,3.04,69178.0,-31668.0,57494.0,-1.43,-31.40,83.11,25.61,142.0,-0.55,0.53,28.26,AR,NTT,CERAMICS
43,JIOFIN,387.0,-2.53,38.0,H-LC,13.87,215565.0,-1841.0,55336.0,-1.01,-0.85,25.67,24.61,48.0,-0.03,1.66,53.51,XY24,BTT,FINANCE
27,GREENPANEL,537.0,193.28,44.0,M-SC,7.12,131004.0,-52074.0,131056.0,-0.79,-28.44,100.04,43.14,230.0,-0.40,1.01,23.23,XY24,NTT,MISC
85,WHIRLPOOL,2270.0,-39.53,60.0,M-SC,2.50,102158.0,10660.0,68088.0,-0.66,11.65,66.65,86.07,223.0,0.16,0.79,48.35,XR,NTT,DURABLES


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,105.37,70.0,H-SC,12.81,129735.0,-12744.0,32265.0,1.22,-8.94,24.87,13.70,163.0,-0.39,1.00,53.12,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,105.55,62.0,M-SC,5.57,88569.0,-12208.0,12302.0,-0.38,-12.11,13.89,0.09,245.0,-0.99,0.68,16.36,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.61,62.0,H-SC,2.40,223605.0,-46062.0,82398.0,-0.09,-17.08,36.85,13.47,138.0,-0.56,1.72,13.10,XY24,NTT,PAINTS
37,INDIGOPNTS,1408.0,114.29,49.0,M-SC,3.02,140480.0,-34079.0,34109.0,0.46,-19.52,24.28,0.02,221.0,-1.00,1.08,22.19,OX40N,NTT,PAINTS
19,CERA,9475.0,-20.05,45.0,H-SC,2.29,113940.0,-29863.0,56605.0,-0.23,-20.77,49.68,18.60,149.0,-0.53,0.88,25.74,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4998.00,-28.04,43.0,H-LC,14.19,286644.0,-59312.0,183166.0,1.01,-17.14,63.90,35.80,1.0,-0.32,2.21,1.55,X40,BTT,IT
39,INFY,2275.00,-16.37,57.0,H-LC,10.13,325144.0,12256.0,159418.0,1.48,3.92,49.03,54.87,3.0,0.08,2.50,10.83,X40,BTT,IT
41,ITC,452.00,-37.92,51.0,H-LC,3.29,200539.0,401.0,20495.0,0.01,0.20,10.22,10.44,4.0,0.02,1.54,5.77,X40,NTT,FMCG
83,VBL,671.64,-15.07,42.0,H-LC,6.37,305698.0,-10144.0,122799.0,0.98,-3.21,40.17,35.67,5.0,-0.08,2.35,10.20,X40N,ATH,FMCG
1,ABB,7934.00,-40.64,49.0,H-LC,12.53,248112.0,-13507.0,132715.0,0.72,-5.16,53.49,45.57,7.0,-0.10,1.91,5.57,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,BLUESTARCO,2326.38,9.48,53.0,H-MC,3.54,189010.0,24340.0,43624.0,0.16,14.78,23.08,41.28,89.0,0.56,1.46,23.94,X40N,ATH,AC
4,ANGELONE,3033.00,9.18,42.0,H-SC,6.37,192995.0,1989.0,55718.0,0.75,1.04,28.87,30.21,157.0,0.04,1.49,22.14,X40N,NTT,FINANCE
84,VOLTAS,1918.49,0.88,59.0,H-MC,3.29,211635.0,19893.0,76146.0,-0.10,10.37,35.98,50.08,99.0,0.26,1.63,17.88,XY25,ATH,AC
30,HAVELLS,2069.16,0.06,62.0,H-MC,1.74,248040.0,309.0,74759.0,0.18,0.12,30.14,30.30,92.0,0.00,1.91,13.43,X40,ATH,ELECTRICAL
24,DABUR,735.00,-2.22,62.0,H-MC,5.58,212901.0,15873.0,73749.0,0.30,8.06,34.64,45.49,102.0,0.22,1.64,22.15,XY24,BTT,FMCG


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,247.44,-44.49,66.0,H-MC,2.81,110362.0,9858.0,20781.0,0.78,9.81,18.83,30.49,79.0,0.47,0.85,45.39,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-39.53,60.0,M-SC,2.50,102158.0,10660.0,68088.0,-0.66,11.65,66.65,86.07,223.0,0.16,0.79,48.35,XR,NTT,DURABLES
36,INDIAMART,4850.92,-51.47,47.0,H-SC,12.10,135574.0,12238.0,121529.0,-0.65,9.92,89.64,108.45,119.0,0.10,1.04,34.49,AR,ATH,MISC
86,WIPRO,420.00,-11.98,51.0,M-LC,6.75,154473.0,3528.0,105922.0,2.73,2.34,68.57,72.51,53.0,0.03,1.19,8.50,XR,NTT,IT
12,BANKINDIA,190.00,-32.92,48.0,H-MC,6.58,173440.0,-6368.0,118772.0,-0.38,-3.54,68.48,62.52,88.0,-0.05,1.34,29.34,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,6711.11,51.0,L-SC,6.08,74715.0,-19095.0,95717.0,0.00,-20.36,128.11,81.67,269.0,-0.20,0.58,44.69,XR,NTT,CERAMICS
3,ALKYLAMINE,4546.37,-5.35,43.0,H-SC,14.66,88081.0,-12882.0,107415.0,0.92,-12.76,121.95,93.63,148.0,-0.12,0.68,31.19,SR,ATH,CHEMICALS
22,COFFEEDAY,80.00,-37.63,61.0,L-SC,42.31,91776.0,-21773.0,61820.0,2.38,-19.17,67.36,35.27,268.0,-0.35,0.71,124.62,XR,NTT,HOTELS
48,MASFIN,398.61,-15.99,60.0,H-SC,13.03,95445.0,-2535.0,24138.0,-0.30,-2.59,25.29,22.05,152.0,-0.11,0.73,38.74,XR,ATH,FINANCE
82,VALIANTORG,838.00,-329.57,58.0,H-SC,8.68,97630.0,-35656.0,120251.0,-3.64,-26.75,123.17,63.47,139.0,-0.30,0.75,63.65,XR,NTT,CHEMICALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.0,-37.63,61.0,L-SC,42.31,91776.0,-21773.0,61820.0,2.38,-19.17,67.36,35.27,268.0,-0.35,0.71,124.62,XR,NTT,HOTELS
60,SAMMAANCAP,326.0,-171.59,64.0,M-SC,3.95,82062.0,-20148.0,113541.0,0.33,-19.71,138.36,91.37,208.0,-0.18,0.63,33.21,XY25,NTT,FINANCE
8,ATULAUTO,844.0,3710.81,63.0,M-SC,7.60,118720.0,-28648.0,80468.0,-0.31,-19.44,67.78,35.16,236.0,-0.36,0.91,21.26,XY24,NTT,AUTO
75,TITAGARH,1548.0,-4.51,62.0,H-SC,7.17,156739.0,-35276.0,114169.0,1.32,-18.37,72.84,41.08,158.0,-0.31,1.21,32.82,XY24,NTT,ENGINEERING
64,SHALBY,327.0,1086.60,75.0,M-SC,2.33,161358.0,-19785.0,65253.0,0.64,-10.92,40.44,25.10,235.0,-0.30,1.24,29.31,XY24,NTT,HEALTHCARE


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.35
1,25,44.52
2,50,75.08


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    40.87
LC    35.37
MC    23.77
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.68
X40      16.04
X40N     13.17
XY25     10.31
XR        9.50
AR        8.33
OX40N     7.77
X200      1.85
MH        1.79
X5K       1.45
SR        1.12
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    26.84
H-SC    24.97
H-MC    20.68
M-SC    14.21
M-LC     7.42
M-MC     2.76
L-SC     1.69
L-LC     1.11
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,12.88,-16.66,75.72
FMCG,12.34,-3.50,38.59
FINANCE,9.06,-18.79,61.51
BANKS,7.70,-14.04,62.46
PAINTS,6.18,-14.87,40.58
MISC,6.03,-17.42,84.73
ELECTRICAL,5.88,-7.70,67.05
AC,3.84,2.73,30.22
AUTO,3.51,-15.15,67.08


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2740984.0,22
AR,1188031.0,9
XR,1060054.0,13
X40,883046.0,10
X40N,565711.0,10
OX40N,449744.0,10
XY25,413156.0,7
SR,198224.0,2
X5K,128849.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2780722.0,28
M-SC,1400984.0,17
H-LC,1378507.0,15
H-MC,1159868.0,15
M-LC,414549.0,5
M-MC,319261.0,2
L-SC,242480.0,3
L-MC,59873.0,1
L-LC,40564.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,920854.0,6
M-SC,XY24,816274.0,7
H-SC,AR,657305.0,5
H-LC,X40,594267.0,5
H-SC,XR,503151.0,6
H-MC,XY24,462568.0,4
H-LC,AR,371157.0,2
M-MC,XY24,319261.0,2
H-SC,X40N,247549.0,4


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
